In [29]:
import os
import requests
import random
import pickle
KEY = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'
KEY2 = 'flLQZGa0kx8opciUA6M7R9iLPwfFPj5yFKKlCiVb'

MOST_RECENT_HR_BILL = 6536 #1490
MOST_RECENT_S_BILL = 3548  #606
CONGRESS = 114

In [35]:
def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

def getSponsors(bill_id, congress, progress = 'introduced'):
    '''Returns (PrimarySponsor, [cosponsors]) for specified bill_id and congress strings. Returns () if faulty'''
    '''progress = 'introduced', 'became law'                                                                '''
    edge = getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/' + bill_id + '/cosponsors.json')
    toReturn = False
    if edge['status'] == 'OK':
        if progress =='introduced':
            toReturn = True
        elif progress == 'became law':
            toReturn = edge['results'][0]['latest_major_action'][:17] == 'Became Public Law'           
                
    if toReturn:
        return (edge['results'][0]['sponsor_id'], [cosponsor['cosponsor_id'] for cosponsor in edge['results'][0]['cosponsors']])
    else:
        return ()
    
def getMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'],0) for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],0) for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)

def getNameIDPairing(congress):
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'], senator['first_name'] + ' ' + senator['last_name'] + '(' + senator['party'] + ')') for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],rep['first_name'] + ' ' + rep['last_name'] + '(' + rep['party'] + ')') for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)
    
    
def buildGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph

def buildFilteredGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress, 'became law')
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress, 'became law')
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph

def buildFilteredByNumActionsGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            num_actions = len(getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/hr{0}.json'.format(i))['results'][0]['actions'])        
            print("has cosponsors")
            if num_actions > 2:
                graph[edge[0]].extend(edge[1])
                print("has some action")
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            num_actions = len(getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/s{0}.json'.format(i))['results'][0]['actions'])        
            print("has cosponsors")
            if num_actions > 2: 
                graph[edge[0]].extend(edge[1])
                print("has some action")
    return graph



def ranker(members, graph, p=.15, iterations=1000000):
    assert len(graph) == len(members)
    for m in members:
        members[m] = 0
    member_list = list(graph.keys())
    current = random.choice(member_list)
    members[current] += 1
    for i in range(iterations):
#         if i % 10 == 0:
#             print("Iteration: {0}".format(i))
        if len(graph[current]) == 0 or random.random() < p:
            current = random.choice(member_list)
#             print("reset")

        else:
            neighbors = graph[current]
            current = neighbors[random.randint(0, len(neighbors)-1)]
#             print("Continue")
        members[current] += 1
    return members


def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [36]:
# members = getMembers(CONGRESS)
graph = buildFilteredByNumActionsGraph(members, CONGRESS)
# names = getNameIDPairing(CONGRESS)
# filteredGraph = buildFilteredGraph(members, CONGRESS)

# save_obj(filteredGraph, 'congress' + str(CONGRESS) + '_filteredGraph')
# save_obj(members, 'congress' + str(CONGRESS) + '_members')
# save_obj(graph, 'congress' + str(CONGRESS) + '_graph')
# save_obj(names, 'congress' + str(CONGRESS) + '_names')

At HR1
At HR2
has cosponsors
has some action
At HR3
has cosponsors
has some action
At HR4
At HR5
has cosponsors
has some action
At HR6
has cosponsors
has some action
At HR7
has cosponsors
has some action
At HR8
has cosponsors
has some action
At HR9
has cosponsors
has some action
At HR10
has cosponsors
has some action
At HR11
At HR12
has cosponsors
has some action
At HR13
At HR14
At HR15
At HR16
At HR17
At HR18
At HR19
At HR20
has cosponsors
has some action
At HR21
has cosponsors
has some action
At HR22
has cosponsors
has some action
At HR23
has cosponsors
has some action
At HR24
has cosponsors
has some action
At HR25
has cosponsors
At HR26
has cosponsors
has some action
At HR27
has cosponsors
At HR28
has cosponsors
has some action
At HR29
has cosponsors
At HR30
has cosponsors
has some action
At HR31
has cosponsors
has some action
At HR32
has cosponsors
At HR33
has cosponsors
has some action
At HR34
has cosponsors
has some action
At HR35
has cosponsors
has some action
At HR36
has cospon

KeyboardInterrupt: 

In [146]:
filteredGraph = load_obj('congress' + str(CONGRESS) + '_filteredGraph')
graph = load_obj('congress' + str(CONGRESS) + '_graph')
members = load_obj('congress' + str(CONGRESS) + '_members')
names = load_obj('congress' + str(CONGRESS) + '_names')

In [27]:
# members = load_obj('congress' + str(CONGRESS) + '_members')
ranking = ranker(members, filteredGraph,.15, 100000)n
member_ranks = [pair[0] for pair in sorted(members.items(), key = lambda item: item[1], reverse=True)]
top_score = members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,members[member_ranks[i]]/top_score, names[member_ranks[i]]))

{'H001056': 0, 'B001245': 0, 'P000449': 0, 'K000383': 0, 'M001183': 0, 'B001243': 0, 'T000465': 0, 'K000384': 0, 'E000295': 0, 'M000404': 0, 'P000610': 0, 'L000578': 0, 'R000588': 0, 'J000290': 0, 'M001189': 0, 'O000168': 0, 'R000582': 0, 'K000188': 0, 'F000451': 0, 'B001291': 0, 'K000362': 0, 'E000294': 0, 'D000618': 0, 'M001163': 0, 'C001098': 0, 'S001179': 0, 'C001106': 0, 'H001060': 0, 'L000569': 0, 'D000622': 0, 'D000197': 0, 'R000577': 0, 'P000605': 0, 'W000796': 0, 'R000584': 0, 'W000810': 0, 'B001284': 0, 'G000568': 0, 'H001046': 0, 'C001108': 0, 'N000002': 0, 'G000559': 0, 'D000621': 0, 'B001269': 0, 'D000563': 0, 'L000577': 0, 'W000798': 0, 'M001193': 0, 'S001194': 0, 'S001141': 0, 'S001184': 0, 'S001196': 0, 'W000187': 0, 'P000034': 0, 'C001077': 0, 'W000821': 0, 'B001282': 0, 'G000556': 0, 'D000619': 0, 'J000296': 0, 'D000623': 0, 'R000570': 0, 'C000127': 0, 'R000487': 0, 'J000293': 0, 'M000355': 0, 'M000639': 0, 'E000288': 0, 'R000409': 0, 'F000461': 0, 'D000607': 0, 'H001

NameError: name 'filteredGraph' is not defined

In [68]:
test = getJSON('https://api.propublica.org/congress/v1/members/N000147/bills/introduced.json')

In [139]:
test2 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6477/cosponsors.json')

In [135]:
test3 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6431/cosponsors.json')

In [136]:
test3 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6431/cosponsors.json')
test3['results'][0]['latest_major_action'][:17] == 'Became Public Law'

True

In [155]:
len([i for i in filteredGraph.values() if i])

187

In [157]:
test4 = load_obj('congress' + str(CONGRESS) + '_filteredGraph')

In [41]:
getJSON('https://api.propublica.org/congress/v1/114/bills/hr21.json')['results'][0]['actions']

[{'datetime': '2015-02-19 00:00:00 UTC',
  'description': 'Referred to the Subcommittee on Environment.'},
 {'datetime': '2015-01-09 00:00:00 UTC',
  'description': 'Referred to the Subcommittee on Energy and Power.'},
 {'datetime': '2015-01-06 00:00:00 UTC',
  'description': 'Referred to House Energy and Commerce'},
 {'datetime': '2015-01-06 00:00:00 UTC',
  'description': 'Referred to House Science, Space, and Technology'}]